In [ ]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

from srai.constants import WGS84_CRS
from srai.neighbourhoods import AdjacencyNeighbourhood
from srai.plotting.folium_wrapper import plot_all_neighbourhood, plot_neighbours, plot_regions
from srai.regionalizers import (
    AdministrativeBoundaryRegionalizer,
    VoronoiRegionalizer,
    geocode_to_region_gdf,
)

## Adjacency Neighbourhood
It can generate neighbourhoods for all geodataframes with touching geometries.

### Real boundaries example - Italy

In [ ]:
it_gdf = geocode_to_region_gdf(query=["R365331"], by_osmid=True)
plot_regions(it_gdf)

In [ ]:
regionalizer = AdministrativeBoundaryRegionalizer(admin_level=4)
it_regions_gdf = regionalizer.transform(it_gdf)

In [ ]:
plot_regions(it_regions_gdf)

In [ ]:
neighbourhood = AdjacencyNeighbourhood(regions=it_regions_gdf)

#### Nearest neighbours

In [ ]:
region_id = "Lazio"
neighbours = neighbourhood.get_neighbours(region_id)
neighbours

In [ ]:
plot_neighbours(it_regions_gdf, region_id, neighbours)

#### Neighbours at a distance

In [ ]:
region_id = "Basilicata"
neighbours = neighbourhood.get_neighbours_at_distance(region_id, 2)
neighbours

In [ ]:
plot_neighbours(it_regions_gdf, region_id, neighbours)

#### Regions without neighbours

In [ ]:
region_id = "Sardinia"
neighbours = neighbourhood.get_neighbours(region_id)
neighbours

In [ ]:
plot_neighbours(it_regions_gdf, region_id, neighbours)

#### Plotting all neighbourhood

In [ ]:
region_id = "Campania"
plot_all_neighbourhood(it_regions_gdf, region_id, neighbourhood)

### Voronoi example - Australia

In [ ]:
au_gdf = geocode_to_region_gdf(query=["R80500"], by_osmid=True)
plot_regions(au_gdf)

In [ ]:
def generate_random_points(shape, n_points=500):
    """Generates random points."""
    minx, miny, maxx, maxy = shape.bounds
    pts = []

    rng = np.random.default_rng()
    while len(pts) < 4:
        randx = rng.uniform(minx, maxx, n_points)
        randy = rng.uniform(miny, maxy, n_points)
        coords = np.vstack((randx, randy)).T

        # use only the points inside the geographic area
        pts = [p for p in list(map(Point, coords)) if p.within(shape)]

    del coords  # not used any more

    return pts

In [ ]:
pts = generate_random_points(au_gdf.geometry[0])

au_seeds_gdf = gpd.GeoDataFrame(
    {"geometry": pts},
    index=list(range(len(pts))),
    crs=WGS84_CRS,
)

In [ ]:
vr = VoronoiRegionalizer(seeds=au_seeds_gdf)
au_result_gdf = vr.transform(gdf=au_gdf)

In [ ]:
folium_map = plot_regions(au_result_gdf, tiles="CartoDB positron")
au_seeds_gdf.explore(
    m=folium_map,
    style_kwds=dict(color="#444", opacity=1, fillColor="#f2f2f2", fillOpacity=1),
    marker_kwds=dict(radius=3),
)

In [ ]:
neighbourhood = AdjacencyNeighbourhood(regions=au_result_gdf)

#### Nearest neighbours

In [ ]:
region_id = 0
neighbours = neighbourhood.get_neighbours(region_id)
neighbours

In [ ]:
plot_neighbours(au_result_gdf, region_id, neighbours)

#### Neighbours at a distance

In [ ]:
region_id = 0
neighbours = neighbourhood.get_neighbours_at_distance(region_id, 3)
neighbours

In [ ]:
plot_neighbours(au_result_gdf, region_id, neighbours)

#### Plotting all neighbourhood

In [ ]:
region_id = 0
plot_all_neighbourhood(au_result_gdf, region_id, neighbourhood)